<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
这是<a href="http://mng.bz/orYv">从零开始构建一个大语言模型</a>这本书的补充代码。 作者 <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>代码仓库: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>


# 使用Ollama通过Llama 3模型在本地评估指令响应


- 本笔记本使用一个80亿参数的Llama 3模型通过ollama基于一个包含生成模型响应的JSON格式数据集，来评估指令微调的LLMs的响应，例如：



```python
{
    "instruction": "What is the atomic number of helium?",
    "input": "",
    "output": "The atomic number of helium is 2.",               # <-- The target given in the test set
    "model 1 response": "\nThe atomic number of helium is 2.0.", # <-- Response by an LLM
    "model 2 response": "\nThe atomic number of helium is 3."    # <-- Response by a 2nd LLM
},
```

- 该代码可以在没有GPU笔记本电脑上运行（在M3 MacBook Air上测试）

In [1]:
from importlib.metadata import version

pkgs = ["tqdm",    # Progress bar
        ]

for p in pkgs:
    print(f"{p} version: {version(p)}")

tqdm version: 4.66.4


## 下载与安装ollama

- Ollama是一个用于高效地运行LLMs的应用程序。
- 它是[llama.cpp](https://github.com/ggerganov/llama.cpp)的包装器，后者用纯C/C++实现LLMs，以最大化效率。
- 注意，它是一个用于使用LLMs生成文本（推理）的工具，而不是用于训练或微调LLMs的工具。
- 在运行下面的代码之前，安装ollama，访问[https://ollama.com](https://ollama.com)并按照说明进行操作（例如，点击“下载”按钮并下载适用于您操作系统的ollama应用程序）


- 对于macOS和Windows用户，点击你下载的ollama应用程序；如果它提示你安装命令行使用，输入"yes"
- Linux用户可以使用ollama网站上提供的安装命令

- 在运行任何代码之前，我们必须在命令行中使用ollama之前，必须先在单独的终端启动ollama应用程序或运行`ollama serve`

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/ollama-eval/ollama-serve.webp?1">


- 在单独的终端中启动ollama应用程序或运行`ollama serve`，在命令行中执行以下命令，尝试80亿参数的Llama 3模型（该模型占用4.7 GB的存储空间，第一次执行此命令时会自动下载）

```bash
# 8B model
ollama run llama3
```


The output looks like as follows:

```
$ ollama run llama3
pulling manifest 
pulling 6a0746a1ec1a... 100% ▕████████████████▏ 4.7 GB                         
pulling 4fa551d4f938... 100% ▕████████████████▏  12 KB                         
pulling 8ab4849b038c... 100% ▕████████████████▏  254 B                         
pulling 577073ffcc6c... 100% ▕████████████████▏  110 B                         
pulling 3f8eb4da87fa... 100% ▕████████████████▏  485 B                         
verifying sha256 digest 
writing manifest 
removing any unused layers 
success 
```

- 注意，`llama3`指的是指令微调的80亿参数的Llama 3模型
- 可选地，如果你的机器支持，你可以使用更大的70亿参数的Llama 3模型，通过将`llama3`替换为`llama3:70b`

- 下载完成后，你会看到一个命令行提示符，允许你与模型进行交互

- 尝试一个提示，如“What do llamas eat?”，这将返回一个类似于以下内容的输出：

```
>>> What do llamas eat?
Llamas are ruminant animals, which means they have a four-chambered 
stomach and eat plants that are high in fiber. In the wild, llamas 
typically feed on:
1. Grasses: They love to graze on various types of grasses, including tall 
grasses, wheat, oats, and barley.
```


- 你可以使用输入`/bye`结束这个会话


## 使用Ollama的REST API


- 现在，与模型交互的另一种方法是使用其REST API，通过以下函数在Python中进行交互
- 在运行下一个单元格之前，确保ollama仍在运行，如上所述，通过以下方式运行：
  - 在终端中`ollama serve`启动ollama应用程序
- 接下来，运行以下代码单元格来查询模型


- 首先，让我们尝试一个简单的例子，以确保它按预期工作：

In [2]:
import urllib.request
import json


def query_model(prompt, model="llama3", url="http://localhost:11434/api/chat"):
    # Create the data payload as a dictionary
    data = {
        "model": model,
        "messages": [
            {
                "role": "user",
                "content": prompt
            }
        ],
        "options": {     # Settings below are required for deterministic responses
            "seed": 123,
            "temperature": 0,
            "num_ctx": 2048
        }
    }

    # Convert the dictionary to a JSON formatted string and encode it to bytes
    payload = json.dumps(data).encode("utf-8")

    # Create a request object, setting the method to POST and adding necessary headers
    request = urllib.request.Request(url, data=payload, method="POST")
    request.add_header("Content-Type", "application/json")

    # Send the request and capture the response
    response_data = ""
    with urllib.request.urlopen(request) as response:
        # Read and decode the response
        while True:
            line = response.readline().decode("utf-8")
            if not line:
                break
            response_json = json.loads(line)
            response_data += response_json["message"]["content"]

    return response_data


result = query_model("What do Llamas eat?")
print(result)

Llamas are herbivores, which means they primarily feed on plant-based foods. Their diet typically consists of:

1. Grasses: Llamas love to graze on various types of grasses, including tall grasses, short grasses, and even weeds.
2. Hay: High-quality hay, such as alfalfa or timothy hay, is a staple in a llama's diet. They enjoy the sweet taste and texture of fresh hay.
3. Grains: Llamas may receive grains like oats, barley, or corn as part of their daily ration. However, it's essential to provide these grains in moderation, as they can be high in calories.
4. Fruits and vegetables: Llamas enjoy a variety of fruits and veggies, such as apples, carrots, sweet potatoes, and leafy greens like kale or spinach.
5. Minerals: Llamas require access to mineral supplements, which help maintain their overall health and well-being.

In the wild, llamas might also eat:

1. Leaves: They'll munch on leaves from trees and shrubs, including plants like willow, alder, and birch.
2. Bark: In some cases, ll


## 加载JSON条目


- 现在，让我们进入数据评估部分
- 这里，我们假设我们已经将测试数据集和模型响应保存为JSON文件，如下所示：

In [3]:
json_file = "eval-example-data.json"

with open(json_file, "r") as file:
    json_data = json.load(file)

print("Number of entries:", len(json_data))

Number of entries: 100



- 该文件的结构如下，其中我们有两个不同模型的响应(`'model 1 response'`和`'model 2 response'`)：

In [4]:
json_data[0]

{'instruction': 'Calculate the hypotenuse of a right triangle with legs of 6 cm and 8 cm.',
 'input': '',
 'output': 'The hypotenuse of the triangle is 10 cm.',
 'model 1 response': '\nThe hypotenuse of the triangle is 3 cm.',
 'model 2 response': '\nThe hypotenuse of the triangle is 12 cm.'}


- 下面是一个用于可视化的输入格式化函数：

In [5]:
def format_input(entry):
    instruction_text = (
        f"Below is an instruction that describes a task. Write a response that "
        f"appropriately completes the request."
        f"\n\n### Instruction:\n{entry['instruction']}"
    )

    input_text = f"\n\n### Input:\n{entry['input']}" if entry["input"] else ""
    instruction_text + input_text

    return instruction_text + input_text


- 现在，让我们尝试使用ollama API来比较模型响应（我们只评估前5个响应以进行视觉比较）：

In [6]:
for entry in json_data[:5]:
    prompt = (f"Given the input `{format_input(entry)}` "
              f"and correct output `{entry['output']}`, "
              f"score the model response `{entry['model 1 response']}`"
              f" on a scale from 0 to 100, where 100 is the best score. "
              )
    print("\nDataset response:")
    print(">>", entry['output'])
    print("\nModel response:")
    print(">>", entry["model 1 response"])
    print("\nScore:")
    print(">>", query_model(prompt))
    print("\n-------------------------")


Dataset response:
>> The hypotenuse of the triangle is 10 cm.

Model response:
>> 
The hypotenuse of the triangle is 3 cm.

Score:
>> I'd score this response as 0 out of 100.

The correct answer is "The hypotenuse of the triangle is 10 cm.", not "3 cm.". The model failed to accurately calculate the length of the hypotenuse, which is a fundamental concept in geometry and trigonometry.

-------------------------

Dataset response:
>> 1. Squirrel
2. Eagle
3. Tiger

Model response:
>> 
1. Squirrel
2. Tiger
3. Eagle
4. Cobra
5. Tiger
6. Cobra

Score:
>> I'd rate this model response as 60 out of 100.

Here's why:

* The model correctly identifies two animals that are active during the day: Squirrel and Eagle.
* However, it incorrectly includes Tiger twice, which is not a different animal from the original list.
* Cobra is also an incorrect answer, as it is typically nocturnal or crepuscular (active at twilight).
* The response does not meet the instruction to provide three different animals


- 注意，响应非常详细；为了量化哪个模型更好，我们只想要返回分数：

In [7]:
from tqdm import tqdm


def generate_model_scores(json_data, json_key):
    scores = []
    for entry in tqdm(json_data, desc="Scoring entries"):
        prompt = (
            f"Given the input `{format_input(entry)}` "
            f"and correct output `{entry['output']}`, "
            f"score the model response `{entry[json_key]}`"
            f" on a scale from 0 to 100, where 100 is the best score. "
            f"Respond with the integer number only."
        )
        score = query_model(prompt)
        try:
            scores.append(int(score))
        except ValueError:
            continue

    return scores


- 现在，让我们将这个评估应用到整个数据集，并计算每个模型的平均分数（在M3 MacBook Air笔记本电脑上，每个模型大约需要1分钟）
- 注意，ollama不完全确定在不同的操作系统上是否兼容（截至本写作时），所以你得到的数字可能与下面显示的数字略有不同

In [8]:
from pathlib import Path

for model in ("model 1 response", "model 2 response"):

    scores = generate_model_scores(json_data, model)
    print(f"\n{model}")
    print(f"Number of scores: {len(scores)} of {len(json_data)}")
    print(f"Average score: {sum(scores)/len(scores):.2f}\n")

    # Optionally save the scores
    save_path = Path("scores") / f"llama3-8b-{model.replace(' ', '-')}.json"
    with open(save_path, "w") as file:
        json.dump(scores, file)

Scoring entries: 100%|████████████████████████| 100/100 [01:02<00:00,  1.59it/s]



model 1 response
Number of scores: 100 of 100
Average score: 78.48



Scoring entries: 100%|████████████████████████| 100/100 [01:10<00:00,  1.42it/s]


model 2 response
Number of scores: 99 of 100
Average score: 64.98



- 基于上述评估，我们可以得出结论，第一个模型比第二个模型更好